In [ ]:
from bs4 import BeautifulSoup

path = 'produced_data/header.xml'

with open(path, 'r', encoding='utf-8') as f:
    def extract_xml_id_and_value_label(xml_data):
        soup = BeautifulSoup(xml_data, 'xml')

        # Get all the person elements.
        persons = soup.find_all('person')

        # Extract the XML IDs and value labels.
        xml_ids_and_value_labels = [(person['xml:id'], person.find('persName').text) for person in persons]

        return xml_ids_and_value_labels
    
    def retrieve_places(xml_data):
        soup = BeautifulSoup(xml_data, 'xml')

        # Get all the person elements.
        places = soup.find_all('place')

        # Extract the XML IDs and value labels.
        xml_ids_and_value_labels = [(place['xml:id'], place.find('placeName').text) for place in places]

        return xml_ids_and_value_labels

# Example usage:

    xml_ids_and_value_labels = retrieve_places(f)

    print(xml_ids_and_value_labels)

[('Urbino', 'Urbino'), ('Europe', 'Europa'), ('Genoa', 'Genova'), ('Nemours', 'Nemours'), ('Greece', 'Grecia'), ('Venice', 'Venezia'), ('Naples', 'Napoli'), ('Bologna', 'Bologna'), ('MCMBologna', 'Museo Civico Medievale di Bologna'), ('Uffizi', 'Uffizi'), ('BSanPetronio', 'basilica di San Petronio'), ('Egypt', 'Egitto'), ('palazzo-salina-amorini', 'Palazzo Bolognini Amorini'), ('Italy', 'Italia'), ('Cremona', 'Cremona'), ('NGLondon', 'National Gallery'), ('Rome', 'Roma'), ('VaticanMuseums', 'Musei Vaticani'), ('Madrid', 'Madrid'), ('PradoMuseum', 'Museo del Prado'), ('Palermo', 'Palermo'), ('Sicily', 'Sicilia'), ('Capri', 'Isola di Capri'), ('Annunziata-Paternò', 'monastero della Santissima Annunziata\n'), ('Imola', 'Imola'), ('Paris', 'Parigi'), ('France', 'Francia'), ('Pisa', 'Pisa'), ('Florence', 'Firenze'), ('HCourt', 'Hampton Court'), ('sanlinfonte', 'chiesa di San Lorenzo in Fonte'), ('accademia-san-luca-roma', 'Accademia nazionale di San Luca'), ('san-domenico-bologna', 'basilic

In [ ]:
from realNameList import personList
from placesList import placeList
import sparql_dataframe
import pandas

#store the people missing from wikidata and the multiple entries
missingpeople=[]
multiplepeople=[]

placeListLabel = []
for i in placeList:
       placeListLabel.append(i[1])


for i[1] in personList[0:5]:

       #result=""

       endpoint = 'https://query.wikidata.org/sparql'

       queryPerson = """
       SELECT ?human ?humanLabel ?genderLabel ?birthDate ?birthPlace ?birthPlaceLabel ?deathDate ?deathPlace ?deathPlaceLabel ?occupation ?occupationLabel WHERE {
       ?human wdt:P31 wd:Q5; # Instances of human (Q5)
              rdfs:label "Beato Angelico"@it. # Label in English
       ?human rdfs:label ?humanLabel. # Get the label
       FILTER(LANG(?humanLabel) = "it") # Filter for English labels
       OPTIONAL{?human wdt:P21 ?gender;
              wdt:P569 ?birthDate;
              wdt:P19 ?birthPlace;
              wdt:P570 ?deathDate;
              wdt:P20 ?deathPlace;
              wdt:P106 ?occupation.
              ?gender rdfs:label ?genderLabel.
              ?birthPlace rdfs:label ?birthPlaceLabel.
              ?deathPlace rdfs:label ?deathPlaceLabel.
              ?occupation rdfs:label ?occupationLabel.
              FILTER(LANG(?genderLabel) = "en") # Filter for English labels
              FILTER(LANG(?birthPlaceLabel) = "en") # Filter for English labels
              FILTER(LANG(?deathPlaceLabel) = "en") # Filter for English labels
              FILTER(LANG(?occupationLabel) = "en") # Filter for English labels
              }

       }
       """

       df = sparql_dataframe.get(endpoint, queryPerson, post='True')

       birthPlaceLabel = df['birthPlaceLabel'][0] 
       deathPlaceLabel = df['deathPlaceLabel'][0]

       #check if there's only one person in wikidata
       if df.empty:
              missingpeople.append(i)
       
       same = df['human'].unique()
       if len(same)>1:
              multiple_entries= same.tolist()
              multiplepeople.append(multiple_entries)



       createrdf = """
       <rdf:Description tei:ref='{0}' rdf:about='{1}'>
              <rdf:type rdf:resource="http://xmlns.com/foaf/0.1/Person"/>
              <rdf:label xml:lan="it">{2}</rdf:label>
              <crm:P61_has_gender>{3}</crm:P61_has_gender>
              <schema:birthDate>{4}</schema:birthDate>
              <scema:deathDate>{5}</scema:deathDate>


       """.format(id=i[0], wikidataentity=df['human'][0], label=i[1], genderLabel=df['genderLabel'][0], birthDate=df['birthDate'][0], deathDate=df['deathDate'][0])
       #add birth place as referenced in the header or as entity in wikidata
       if birthPlaceLabel in placeListLabel:
              birthPlaceLabelId= placeList[0]
       #ho dei dubbi sul primo caso
              addbirthplace = createrdf + """

              <schema:birthPlace xml:id="'%(birthPlaceLabelId)s'"/>

              """%birthPlaceLabelId

       else:
              addbirthplace = createrdf + """
              <schema:birthPlace rdf:resource="%()s"
       """%df['birthPlace'][0]

       #add death place as referenced in the header or as entity in wikidata

       if deathPlaceLabel in placeListLabel:
              deathPlaceLabelId= placeList[0]
       #ho dei dubbi sul primo caso
              adddeathplace = addbirthplace + """

              <schema:birthPlace xml:id="'%(deathPlaceLabelId)s'"/>

              """%deathPlaceLabelId

       else:
              adddeathplace= addbirthplace + """
              <schema:deathPlace rdf:resource="%()s"
       """%df['deathPlace'][0]
              
       result = adddeathplace + """
       </rdf:Description>
       """

result, missingpeople, multiplepeople      


,human,humanLabel,genderLabel,birthDate,birthPlace,birthPlaceLabel,deathDate,deathPlace,deathPlaceLabel,occupation,occupationLabel
0,http://www.wikidata.org/entity/Q5664,Beato Angelico,male,1390-01-01T00:00:00Z,http://www.wikidata.org/entity/Q82880,Vicchio,1455-02-18T00:00:00Z,http://www.wikidata.org/entity/Q220,Rome,http://www.wikidata.org/entity/Q14623005,architectural draftsperson
1,http://www.wikidata.org/entity/Q5664,Beato Angelico,male,1390-01-01T00:00:00Z,http://www.wikidata.org/entity/Q82880,Vicchio,1455-02-18T00:00:00Z,http://www.wikidata.org/entity/Q220,Rome,http://www.wikidata.org/entity/Q998628,illuminator
2,http://www.wikidata.org/entity/Q5664,Beato Angelico,male,1390-01-01T00:00:00Z,http://www.wikidata.org/entity/Q82880,Vicchio,1455-02-18T00:00:00Z,http://www.wikidata.org/entity/Q220,Rome,http://www.wikidata.org/entity/Q1028181,painter


In [ ]:
#query for places
for i in placeList[:5]:
       placeListLabel.append(i[1])
       
       endpoint = 'https://query.wikidata.org/sparql'

       queryPlace= """SELECT ?place ?placeLabel ?coordinate  WHERE {
       ?place wdt:P31/wdt:P279* wd:Q2221906; # Instances of place (Q2221906) or its subclasses
              rdfs:label "Porta Pia"@it; # Label in English
              wdt:P625 ?coordinate.
       ?place rdfs:label ?placeLabel. # Get the label
       FILTER(LANG(?placeLabel) = "en") # Filter for English labels
       }
       """

       dfplace = sparql_dataframe.get(endpoint, queryPlace, post='True')

       createrdf ="""
 <rdf:Description tei:ref="{1}" rdf:about="">
       <rdf:type rdf:resource="{2}"/>
       <rdfs:label xml:lang="en">{3}</rdfs:label>
</rdf:Description>

""".format(id=i[0], wikidataentity = dfplace['place'][0], label = i[1], )

       

